In [2]:
import warnings
warnings.filterwarnings('ignore')

import geopandas
import r5py
import shapely
import pandas as pd
import geopandas as gpd
import numpy as np
import datetime
import copy

import os
import snman
from snman import osmnx_customized as oxc
from snman.constants import *

PERIMETER = '_accessibility_debug'

# Set these paths according to your own setup
data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
outputs_path = os.path.join(data_directory, 'outputs', PERIMETER)

CRS_internal = 2056      # for Zurich

In [4]:
after_tt = snman.io.import_geofile_to_gdf(
    os.path.join(outputs_path, 'tt_after_edges.gpkg')
)

In [3]:
before_tt = snman.io.import_geofile_to_gdf(
    os.path.join(outputs_path, 'tt_before_edges.gpkg')
)
before_tt

,u,v,key,OSM_ID,FreeFlowTTCar,car_7:00,car_18:00,N_cars_7:00,N_cars_18:00,highway,length,lanes,oneway,geometry
0,0,11,0,7749004,74.62,74.62,81.00,0,5,residential,310.929535,M>*3.0,yes,"LINESTRING (2671251.849 1242229.885, 2671236.1..."
1,0,17108,0,7749021,15.39,22.00,22.00,25,147,tertiary,106.850626,M>*3.0,yes,"LINESTRING (2671251.849 1242229.885, 2671202.1..."
2,0,17472,0,7749008,22.62,23.02,38.35,166,141,tertiary,1082.720913,M>*3.0,yes,"LINESTRING (2671251.849 1242229.885, 2671388.3..."
3,0,17472,1,7749008,26.10,33.00,61.92,167,141,tertiary,1082.720913,M>*3.0,yes,"LINESTRING (2671251.849 1242229.885, 2671388.3..."
4,1,12832,0,7701086,22.48,29.00,22.48,3,0,residential,93.677650,M>*3.0,yes,"LINESTRING (2683253.128 1261522.646, 2683332.6..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143697,56234,29985,0,7777867,9.20,10.00,10.00,1108,1451,motorway,813.948154,M>*3.0,yes,"LINESTRING (2677103.268 1232195.346, 2677196.8..."
143698,56234,29985,1,7777867,15.22,16.00,16.00,1115,1452,motorway,813.948154,M>*3.0,yes,"LINESTRING (2677103.268 1232195.346, 2677196.8..."
143699,56235,23,0,7740686,46.18,53.00,53.00,4,2,residential,192.410583,M>*3.0,yes,"LINESTRING (2641479.498 1244490.799, 2641516.1..."
143700,56235,56090,0,7740684,37.19,43.00,44.00,1,2,residential,154.958620,M>*3.0,yes,"LINESTRING (2641479.498 1244490.799, 2641476.1..."


In [6]:
after_tt

,u,v,key,OSM_ID,FreeFlowTTCar,car_7:00,car_18:00,N_cars_7:00,N_cars_18:00,highway,length,lanes,oneway,geometry
0,0,3,0,7701054,22.34,29.01,28.46,78,112,residential,186.182650,M>*3.0,yes,"LINESTRING (2701971.251 1264550.449, 2702085.5..."
1,0,19,0,7701051,6.77,13.07,13.49,27,75,residential,56.407083,M>*3.0,yes,"LINESTRING (2701971.251 1264550.449, 2702001.9..."
2,0,3636,0,7701053,9.81,16.01,16.06,96,108,residential,81.749086,M>*3.0,yes,"LINESTRING (2701971.251 1264550.449, 2701945.5..."
3,0,5858,0,7701052,13.91,20.00,20.32,13,19,residential,115.886530,M>*3.0,yes,"LINESTRING (2701971.251 1264550.449, 2701866.5..."
4,1,2,0,7715684,9.20,16.00,16.00,2,5,residential,76.629765,M>*3.0,yes,"LINESTRING (2699643.283 1265696.181, 2699645.4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143330,55376,16469,0,7591243,65.50,65.50,65.50,0,0,residential,272.917652,M>*3.0,yes,"LINESTRING (2699733.512 1237068.932, 2699704.4..."
143331,55376,29525,0,7591241,40.16,40.16,46.00,0,1,residential,167.314346,M>*3.0,yes,"LINESTRING (2699733.512 1237068.932, 2699829.5..."
143332,55377,6510,0,7576940,82.79,26.53,21.08,17,13,residential,344.957068,M>*3.0,yes,"LINESTRING (2684442.229 1231517.003, 2684264.8..."
143333,55378,6641,0,7767687,10.19,11.00,11.00,1420,852,motorway,283.145154,M>*3.0,yes,"LINESTRING (2674000.659 1247776.848, 2674040.4..."


In [14]:
before_tt['pseudo_id'] = before_tt.apply(
    lambda row: f"{row.geometry}_{row['highway']}_{row['key']}",
    axis=1
)
after_tt['pseudo_id'] = after_tt.apply(
    lambda row: f"{row.geometry}_{row['highway']}_{row['key']}",
    axis=1
)

In [15]:
after_tt

,u,v,key,OSM_ID,FreeFlowTTCar,car_7:00,car_18:00,N_cars_7:00,N_cars_18:00,highway,length,lanes,oneway,geometry,pseudo_id
0,0,3,0,7701054,22.34,29.01,28.46,78,112,residential,186.182650,M>*3.0,yes,"LINESTRING (2701971.251 1264550.449, 2702085.5...",LINESTRING (2701971.250515788 1264550.44906123...
1,0,19,0,7701051,6.77,13.07,13.49,27,75,residential,56.407083,M>*3.0,yes,"LINESTRING (2701971.251 1264550.449, 2702001.9...",LINESTRING (2701971.250515788 1264550.44906123...
2,0,3636,0,7701053,9.81,16.01,16.06,96,108,residential,81.749086,M>*3.0,yes,"LINESTRING (2701971.251 1264550.449, 2701945.5...",LINESTRING (2701971.250515788 1264550.44906123...
3,0,5858,0,7701052,13.91,20.00,20.32,13,19,residential,115.886530,M>*3.0,yes,"LINESTRING (2701971.251 1264550.449, 2701866.5...",LINESTRING (2701971.250515788 1264550.44906123...
4,1,2,0,7715684,9.20,16.00,16.00,2,5,residential,76.629765,M>*3.0,yes,"LINESTRING (2699643.283 1265696.181, 2699645.4...",LINESTRING (2699643.283163938 1265696.18108877...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143330,55376,16469,0,7591243,65.50,65.50,65.50,0,0,residential,272.917652,M>*3.0,yes,"LINESTRING (2699733.512 1237068.932, 2699704.4...",LINESTRING (2699733.511852382 1237068.93164802...
143331,55376,29525,0,7591241,40.16,40.16,46.00,0,1,residential,167.314346,M>*3.0,yes,"LINESTRING (2699733.512 1237068.932, 2699829.5...",LINESTRING (2699733.511852382 1237068.93164802...
143332,55377,6510,0,7576940,82.79,26.53,21.08,17,13,residential,344.957068,M>*3.0,yes,"LINESTRING (2684442.229 1231517.003, 2684264.8...",LINESTRING (2684442.2291714326 1231517.0030021...
143333,55378,6641,0,7767687,10.19,11.00,11.00,1420,852,motorway,283.145154,M>*3.0,yes,"LINESTRING (2674000.659 1247776.848, 2674040.4...",LINESTRING (2674000.658519982 1247776.84808859...


In [27]:
m = pd.merge(before_tt, after_tt[['car_7:00', 'car_18:00', 'N_cars_7:00', 'N_cars_18:00', 'pseudo_id', 'OSM_ID']], on='pseudo_id', suffixes=['_before', '_after'])
m

,u,v,key,OSM_ID_before,FreeFlowTTCar,car_7:00_before,car_18:00_before,N_cars_7:00_before,N_cars_18:00_before,highway,length,lanes,oneway,geometry,pseudo_id,car_7:00_after,car_18:00_after,N_cars_7:00_after,N_cars_18:00_after,OSM_ID_after
0,0,11,0,7749004,74.62,74.62,81.00,0,5,residential,310.929535,M>*3.0,yes,"LINESTRING (2671251.849 1242229.885, 2671236.1...",LINESTRING (2671251.8490574374 1242229.8852307...,74.62,74.62,0,0,7749585
1,0,17108,0,7749021,15.39,22.00,22.00,25,147,tertiary,106.850626,M>*3.0,yes,"LINESTRING (2671251.849 1242229.885, 2671202.1...",LINESTRING (2671251.8490574374 1242229.8852307...,22.00,22.00,25,112,7749602
2,0,17472,0,7749008,22.62,23.02,38.35,166,141,tertiary,1082.720913,M>*3.0,yes,"LINESTRING (2671251.849 1242229.885, 2671388.3...",LINESTRING (2671251.8490574374 1242229.8852307...,33.00,33.04,59,112,7749589
3,0,17472,1,7749008,26.10,33.00,61.92,167,141,tertiary,1082.720913,M>*3.0,yes,"LINESTRING (2671251.849 1242229.885, 2671388.3...",LINESTRING (2671251.8490574374 1242229.8852307...,23.00,23.00,58,112,7749589
4,1,12832,0,7701086,22.48,29.00,22.48,3,0,residential,93.677650,M>*3.0,yes,"LINESTRING (2683253.128 1261522.646, 2683332.6...",LINESTRING (2683253.1282696715 1261522.6462885...,29.00,22.48,3,0,7701614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134210,56234,29985,0,7777867,9.20,10.00,10.00,1108,1451,motorway,813.948154,M>*3.0,yes,"LINESTRING (2677103.268 1232195.346, 2677196.8...",LINESTRING (2677103.2675720016 1232195.3459392...,16.00,16.00,1102,1205,7778448
134211,56234,29985,1,7777867,15.22,16.00,16.00,1115,1452,motorway,813.948154,M>*3.0,yes,"LINESTRING (2677103.268 1232195.346, 2677196.8...",LINESTRING (2677103.2675720016 1232195.3459392...,10.00,10.00,1095,1203,7778448
134212,56235,23,0,7740686,46.18,53.00,53.00,4,2,residential,192.410583,M>*3.0,yes,"LINESTRING (2641479.498 1244490.799, 2641516.1...",LINESTRING (2641479.4976011836 1244490.7990828...,53.00,53.00,3,1,7741267
134213,56235,56090,0,7740684,37.19,43.00,44.00,1,2,residential,154.958620,M>*3.0,yes,"LINESTRING (2641479.498 1244490.799, 2641476.1...",LINESTRING (2641479.4976011836 1244490.7990828...,43.00,37.19,1,0,7741265


In [28]:
for hour in ['7:00', '18:00']:
    m[f"N_cars_{hour}_diff"] = m[f"N_cars_{hour}_after"] - m[f"N_cars_{hour}_before"]
    m[f"car_{hour}_diff"] = m[f"car_{hour}_after"] - m[f"car_{hour}_before"]
m

,u,v,key,OSM_ID_before,FreeFlowTTCar,car_7:00_before,car_18:00_before,N_cars_7:00_before,N_cars_18:00_before,highway,...,pseudo_id,car_7:00_after,car_18:00_after,N_cars_7:00_after,N_cars_18:00_after,OSM_ID_after,N_cars_7:00_diff,car_7:00_diff,N_cars_18:00_diff,car_18:00_diff
0,0,11,0,7749004,74.62,74.62,81.00,0,5,residential,...,LINESTRING (2671251.8490574374 1242229.8852307...,74.62,74.62,0,0,7749585,0,0.00,-5,-6.38
1,0,17108,0,7749021,15.39,22.00,22.00,25,147,tertiary,...,LINESTRING (2671251.8490574374 1242229.8852307...,22.00,22.00,25,112,7749602,0,0.00,-35,0.00
2,0,17472,0,7749008,22.62,23.02,38.35,166,141,tertiary,...,LINESTRING (2671251.8490574374 1242229.8852307...,33.00,33.04,59,112,7749589,-107,9.98,-29,-5.31
3,0,17472,1,7749008,26.10,33.00,61.92,167,141,tertiary,...,LINESTRING (2671251.8490574374 1242229.8852307...,23.00,23.00,58,112,7749589,-109,-10.00,-29,-38.92
4,1,12832,0,7701086,22.48,29.00,22.48,3,0,residential,...,LINESTRING (2683253.1282696715 1261522.6462885...,29.00,22.48,3,0,7701614,0,0.00,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134210,56234,29985,0,7777867,9.20,10.00,10.00,1108,1451,motorway,...,LINESTRING (2677103.2675720016 1232195.3459392...,16.00,16.00,1102,1205,7778448,-6,6.00,-246,6.00
134211,56234,29985,1,7777867,15.22,16.00,16.00,1115,1452,motorway,...,LINESTRING (2677103.2675720016 1232195.3459392...,10.00,10.00,1095,1203,7778448,-20,-6.00,-249,-6.00
134212,56235,23,0,7740686,46.18,53.00,53.00,4,2,residential,...,LINESTRING (2641479.4976011836 1244490.7990828...,53.00,53.00,3,1,7741267,-1,0.00,-1,0.00
134213,56235,56090,0,7740684,37.19,43.00,44.00,1,2,residential,...,LINESTRING (2641479.4976011836 1244490.7990828...,43.00,37.19,1,0,7741265,0,0.00,-2,-6.81


In [30]:
snman.io.export_gdf(
    m,
    os.path.join(outputs_path, 'tt_diff_edges.gpkg')
)

In [34]:
sum(before_tt['N_cars_7:00'])

6372835

In [35]:
sum(after_tt['N_cars_7:00'])

6229721